<a href="https://colab.research.google.com/github/srinivasrdhkrshnn/CS6910-Assignment-4/blob/main/Contrastive_Divergence_Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb

***Importing Libraries and Load Data***

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.datasets import mnist
from sklearn.manifold import TSNE
import pickle
import wandb

# load dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
class_type = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] 

proj_name='CS6910 ASSIGNMENT 4'

***Modify and Restructure Data***

In [ ]:
# Initializations

# Data
X_train = np.array(x_train.reshape(x_train.shape[0], 784,1))         # reshape 2-D data to 1-D
X_test  = np.array(x_test.reshape(x_test.shape[0], 784,1))           # reshape 2-D data to 1-D
X_train = (X_train > 126) * 1                                        # convert the real valued data into binary data, using a threshold of 127
X_test  = (X_test > 126) * 1                                         # convert the real valued data into binary data, using a threshold of 127

X_val = X_train[-15000:]                                             # validation set input (to train Classifier)
X_train = X_train[0:45000]                                           # training set input (to train RBM)

Y_train = np.zeros([len(y_train),10,1])
Y_test = np.zeros([len(y_test),10,1])

for i in range(len(y_train)):                                        # convert y from just a class number to one hot vector (10x1)

      y = np.zeros([10, 1])
      y[y_train[i]] = 1.0
      Y_train[i] = y

for i in range(len(y_test)):                                         # convert y from just a class number to one hot vector (10x1)

      y = np.zeros([10, 1])
      y[y_test[i]] = 1.0
      Y_test[i] = y                                                  # test set output

Y_val = Y_train[-15000:]                                             # validation set output
Y_train = Y_train[0:45000]                                           # training set output

***Hyperparameters and Basic variables***

In [ ]:
n_visible = X_train.shape[1]                                         # number of visible neurons
n_train_examples = X_train.shape[0]                                  # number of training data
n_val_examples = X_val.shape[0]                                      # number of validation data
n_test_examples = X_test.shape[0]                                    # number of test data
n_hidden = 80                                                        # number of hidden neurons
n_class = 10



***Parameter Initialization (Weights and Biases)***

In [ ]:
def rbm_param_init() :                                                                        # Function to randomly initialize the RBM network parameters 
	
  rbm_parameters = {}
  rbm_parameters["W"] = np.random.randn(n_hidden, n_visible)*np.sqrt(6./(n_visible + n_hidden))   # Xavier Initialization of weights
  rbm_parameters["h_bias"] = np.zeros((n_hidden,1),dtype=np.float64)                                                     
  rbm_parameters["v_bias"] = np.zeros((n_visible,1),dtype=np.float64)

  return rbm_parameters

def classifier_param_init() :                                                                 # Function to randomly initialize the Classifier network parameters

  classifier = {}
  classifier["W"] = np.random.randn(n_class, n_hidden)*np.sqrt(6./(n_class + n_hidden))       # Xavier Initialization of weights
  classifier["b"] = np.zeros((n_class,1),dtype=np.float64)

  return classifier

def sigmoid(x) :                                                                              # RBM hidden layer activation function 
	
  return 1.0/(1.0+np.exp(-x))

def softmax(x):                                                                               # Output activation function
    
  return np.exp(x) / np.sum(np.exp(x))  

***RBM and Classifier Trainer***

In [ ]:
def rbm_train(X_train,parameters,k,learning_rate) :                                     # Function to train the RBM
	
  W = parameters["W"]
  h_bias = parameters["h_bias"]
  v_bias = parameters["v_bias"]
  
  for i in range(n_train_examples) :
      v_sample = X_train[i]
      v_init   = X_train[i]

      for t in range(k) :                                                                 # Markov Chain - Loop
          h_given_v = sigmoid(np.dot(W,v_sample)+h_bias)                                  # Evaluate p(h|v)
          h_sample = np.random.binomial(1,h_given_v)                                      # Convert to 0's and 1's assuming binomial distribution p(h|v) 
          v_given_h = sigmoid(np.dot(np.transpose(W),h_sample)+v_bias)                    # Evaluate p(v|h)
          v_sample = np.random.binomial(1,v_given_h)                                      # Convert to 0's and 1's assuming binomial distribution p(v|h)

      # Update Rule
      W = W + learning_rate*(np.dot(sigmoid(np.dot(W,v_init)+h_bias),np.transpose(v_init)) - np.dot(sigmoid(np.dot(W,v_sample)+h_bias),np.transpose(v_sample)))
      v_bias = v_bias + learning_rate*(v_init-v_sample)
      h_bias = h_bias + learning_rate*(sigmoid(np.dot(W,v_init)+h_bias) - sigmoid(np.dot(W,v_sample)+h_bias))

  parameters["W"] = W
  parameters["h_bias"] = h_bias
  parameters["v_bias"] = v_bias
  print("Training Complete")

  return parameters


def get_hidden(x,parameters) :                                                                # function to get the hidden representation of the test data
   
    W = parameters["W"]
    h_bias = parameters["h_bias"]
    hidden_prob = sigmoid(np.dot(W,x)+h_bias)
    hidden_rep = np.random.binomial(1,hidden_prob)
    
    return hidden_rep

def classifier_train(X,Y,rbm_parameters,classifier_param,classifier_epochs,learning_rate)  :  # Function to train Classifier
    
    W = classifier_param["W"]
    b = classifier_param["b"]
    
    for epoch in range(classifier_epochs) :

        for i in range(n_val_examples) :

            # feed forward
            hidden_rep = get_hidden(X[i],rbm_parameters)                                      # Obtain hidden representation for the given sample
            pre_output = np.dot(W,hidden_rep)+b
            y_hat = softmax(pre_output)
            # backpropogate
            dW = np.dot(-(Y[i]-y_hat),np.transpose(hidden_rep))
            db = -(Y[i]-y_hat)
            # Update Classifier weights
            W = W - learning_rate*dW
            b = b - learning_rate*db

    classifier_param["W"] = W
    classifier_param["b"] = b 
    
    return classifier_param   


In [ ]:
def rbm_classifier(X_train,Y_train,X_val,Y_val,X_test,Y_test,rbm_epochs,classifier_epochs,k,n_hidden,learning_rate) :

  classifier_param = classifier_param_init() 
  rbm_parameters = rbm_param_init()

  for j in range(rbm_epochs) :

     rbm_parameters = rbm_train(X_train,rbm_parameters,k,learning_rate)                                                             # Train RBM for one epoch
     classifier_param = classifier_train(X_val,Y_val,rbm_parameters,classifier_param,classifier_epochs,learning_rate)               # Train the classifier with hidden representation of RBM
     
     accuracy = 0.0
     loss = 0.0 
     # Evaluate accuracy and loss over test data

     for i in range(n_test_examples) :

        h = get_hidden(X_test[i],rbm_parameters)
        y_hat = softmax(np.dot(classifier_param["W"],h)+classifier_param["b"])

        if y_hat.argmax()==Y_test[i].argmax():
            accuracy = accuracy + 1
        loss = loss + -1*np.sum(np.multiply(y,np.log(y_hat)))

     accuracy = accuracy/n_test_examples
     loss = loss/n_test_examples 
     print("Epoch : " + str(j)+" Accuracy : "+ str(accuracy)+" Loss : "+str(loss))
    # wandb.log({"Accuracy":accuracy,"Loss":loss,"Epoch":j})

  return rbm_parameters,classifier_param
      

	

In [ ]:
def train():
  hyperparameter_defaults=dict(
        rbm_epochs = 20,
        classifier_epochs = 1,
        k = 2,
        n_hidden = 256,
        learning_rate = 0.001                                                                    
        )

  wandb.init(config=hyperparameter_defaults)

  config=wandb.config
  rbm_parameters,classifier_param = rbm_classifier(X_train,Y_train,X_val,Y_val,X_test,Y_test,config.rbm_epochs,config.classifier_epochs,config.k,config.n_hidden,config.learning_rate)




In [ ]:
def sweeper(sweep_config,proj_name):
  sweep_id = wandb.sweep(sweep_config,project=proj_name,entity='cs6910krsrd',)
  wandb.agent(sweep_id,train,project=proj_name,entity='cs6910krsrd',)

In [ ]:
#sweep dictionary
sweep_config={
    'method':'bayes',
    'metric':{
        'name':'accuracy',
        'goal':'maximize'},

}

parameters_dict={
    
    'rbm_epochs':{
      'values':[20]  
    },
    'classifier_epochs':{
        'values':[1]
    },
    'k':{
        'values':[20]
    },
    'n_hidden':{
        'values':[64]      #,128,256]
    },
    'learning_rate':{
        'values':[0.01]
    }
    
}

sweep_config['parameters']=parameters_dict

In [ ]:
sweeper(sweep_config,proj_name)

***Training over Best Model***

We train the RBM-Classifier Model over the estimated best hyperparameters (obtained from WandB sweep) and present the evolution of Accuracy and Loss over each epoch.

In [ ]:
rbm_epochs = 20
classifier_epochs = 1
k = 2
n_hidden = 256
learning_rate = 0.001

rbm_parameters,classifier_param = rbm_classifier(X_train,Y_train,X_val,Y_val,X_test,Y_test,rbm_epochs,classifier_epochs,k,n_hidden,learning_rate)


Training Complete
Epoch : 0 Accuracy : 0.6791 Loss : 4.872227434260146
Training Complete
Epoch : 1 Accuracy : 0.7141 Loss : 5.474135236859016
Training Complete
Epoch : 2 Accuracy : 0.7306 Loss : 5.778316353384354
Training Complete
Epoch : 3 Accuracy : 0.735 Loss : 6.010077383480011
Training Complete
Epoch : 4 Accuracy : 0.7462 Loss : 6.059958613596309
Training Complete
Epoch : 5 Accuracy : 0.7449 Loss : 6.32218613801851
Training Complete
Epoch : 6 Accuracy : 0.7534 Loss : 6.505518380290844
Training Complete
Epoch : 7 Accuracy : 0.7594 Loss : 6.601565083097921
Training Complete
Epoch : 8 Accuracy : 0.7629 Loss : 6.689751352389103
Training Complete
Epoch : 9 Accuracy : 0.7682 Loss : 6.819843051875718
Training Complete
Epoch : 10 Accuracy : 0.7665 Loss : 7.005095854865377
Training Complete
Epoch : 11 Accuracy : 0.7661 Loss : 7.033121402890481
Training Complete
Epoch : 12 Accuracy : 0.7702 Loss : 7.083993205769119
Training Complete
Epoch : 13 Accuracy : 0.7709 Loss : 7.195458260839693
Trai

***Saving Parameters***

In [ ]:
MODEL_NAME ="Ass4_Model"
RBM_DIR="rbm_param.pkl"
CLASS_DIR="class_param.pkl"


conf_dict={
    'rbm_epochs':rbm_epochs,
    'classifier_epochs':classifier_epochs,
    'k':k,
    'n_hidden':n_hidden,
    'learning_rate':learning_rate
}


rbm_file=open(RBM_DIR,"wb")
pickle.dump(rbm_parameters,rbm_file)
rbm_file.close()

class_file=open(CLASS_DIR,"wb")
pickle.dump(classifier_param,class_file)
class_file.close()

run = wandb.init(project=proj_name, job_type="save_model",entity='cs6910krsrd',)

rbm_artifact = wandb.Artifact(
          MODEL_NAME+"rbm", type="model",
          description="rbm_parameters",
          metadata=conf_dict)


classifier_artifact = wandb.Artifact(
          MODEL_NAME+"class", type="model",
          description="classifier_param",
          metadata=conf_dict)

rbm_artifact.add_file(RBM_DIR)
classifier_artifact.add_file(CLASS_DIR)

run.log_artifact(rbm_artifact)
run.log_artifact(classifier_artifact)

run.finish()

***Loading Parameters***

In [ ]:
run = wandb.init(project=proj_name, job_type="load_model")


rbm_at = run.use_artifact('cs6910krsrd/CS6910 ASSIGNMENT 4/Ass4_Modelrbm:v1', type='model')                         #load different versions of model
MODEL_DIR= rbm_at.download()

rbm_file=open(MODEL_DIR+"/"+RBM_DIR,"rb")
rbm_parameters = pickle.load(rbm_file)
rbm_file.close()

class_at = run.use_artifact('cs6910krsrd/CS6910 ASSIGNMENT 4/Ass4_Modelclass:v1', type='model')
MODEL_DIR= class_at.download()

class_file=open(MODEL_DIR+"/"+CLASS_DIR,"rb")
classifier_param = pickle.load(class_file)
class_file.close()

run.finish()


***Contrastive Divergence Results***

For the best RBM-Classifier Model, the hyperparameters are 
    

*   n_hidden neurons = 256
*   number of Markov steps = 2
*   learning rate = 0.01

The accuracy achieved over test-data is 78.25% after 20 epochs of training RBM-Classifier Model.





***Visualising Hidden Representation using t-SNE Plot*** 

In [ ]:
from sklearn.manifold import TSNE

hidden_reps = []

for i in range(n_test_examples) :
  
    h = get_hidden(X_test[i],rbm_parameters)
    hidden_reps.append(np.array(h))

print(np.shape(hidden_reps))

hidden_reps = np.array(hidden_reps)
hidden_reps = np.resize(hidden_reps, (n_test_examples,256))
embedded = TSNE(n_components = 2, early_exaggeration = 7).fit_transform(hidden_reps)           # n_components = Dimension of the embedded space (here 2D).

vis_x = embedded[:,0]
vis_y = embedded[:,1]


In [ ]:
run = wandb.init(project=proj_name,entity='cs6910krsrd',)

path_to_img = "./embed.png" 

plt.figure(figsize = (20,16))
plt.title("Embeddings for testing examples")
plt.xlabel("X")
plt.ylabel("Y")
plt.scatter(vis_x[:10000], vis_y[:10000], c = y_test[:10000], cmap = plt.cm.get_cmap("jet", 10))
plt.colorbar(ticks = range(max(y_test)))
plt.savefig ( path_to_img )
plt.show()
class_type = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat','Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
im = plt.imread(path_to_img)

# Log image(s)
wandb.log({"t-SNE Plot": [wandb.Image(im, caption="plot")]})

run.finish() 

In [ ]:
run = wandb.init(project=proj_name,entity='cs6910krsrd',)

print("Sample Images for each Class :")
class_list=list()

for i in range(10):
  plt.subplot(2,5,i+1)
  for j in range(len(y_train)):
    if y_train[j] == i :
        wandb.log({"Sample Images per Class": [wandb.Image(x_train[j], caption=class_type[y_train[j]])]})
        class_list.append(class_type[y_train[j]])
        break

run.finish()  

***Visualise the samples generated from the distribution***

In [ ]:
  run = wandb.init(project=proj_name,entity='cs6910krsrd',)


  rbm_parameters = rbm_param_init()
  rbm_epochs = 10

  W = rbm_parameters["W"]
  h_bias = rbm_parameters["h_bias"]
  v_bias = rbm_parameters["v_bias"]

  for j in range(rbm_epochs) :

      if (j==0) or (j==9) :
          plt.figure(figsize = (10,8))
          subplot_no = 1
          
      for i in range(n_train_examples) :
          v_sample = X_train[i]
          v_init   = X_train[i]

          if ((j==0) or (j==9)) and ((i%(701) == 0)) and (subplot_no <= 64):

            hidden = np.random.binomial(1,sigmoid(np.dot(W,v_sample)+h_bias))
            img_reconstruct = np.random.binomial(1,sigmoid(np.dot(np.transpose(W),hidden)+v_bias))
            img = np.reshape(img_reconstruct, (28,28))
            plt.subplot(8,8,subplot_no)
            plt.imshow(img, cmap = "gray")
            plt.axis("off")
            subplot_no += 1

          for t in range(k) :                                                                # Markov Chain - Loop
             h_given_v = sigmoid(np.dot(W,v_sample)+h_bias)                                  # Evaluate p(h|v)
             h_sample = np.random.binomial(1,h_given_v)                                      # Convert to 0's and 1's assuming binomial distribution p(h|v) 
             v_given_h = sigmoid(np.dot(np.transpose(W),h_sample)+v_bias)                    # Evaluate p(v|h)
             v_sample = np.random.binomial(1,v_given_h)                                      # Convert to 0's and 1's assuming binomial distribution p(v|h)

          # Update Rule
          W = W + learning_rate*(np.dot(sigmoid(np.dot(W,v_init)+h_bias),np.transpose(v_init)) - np.dot(sigmoid(np.dot(W,v_sample)+h_bias),np.transpose(v_sample)))
          v_bias = v_bias + learning_rate*(v_init-v_sample)
          h_bias = h_bias + learning_rate*(sigmoid(np.dot(W,v_init)+h_bias) - sigmoid(np.dot(W,v_sample)+h_bias))


      if (j==0) or (j==9) :
        wandb.log({"Sample Visualisation": plt})


      plt.show()

  rbm_parameters["W"] = W
  rbm_parameters["h_bias"] = h_bias
  rbm_parameters["v_bias"] = v_bias
  print("Training Complete")

  run.finish()
